Importing all the reqiured modules

In [ ]:
#Installing transformers and pydub
!pip install transformers
!pip install pydub
!pip install webrtcvad

#Importing ML and other modules
import matplotlib.pyplot as plt
import math
import os
import time
import operator
from collections import defaultdict
import sklearn
import keras
import csv
import numpy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from IPython.display import Audio
import pydub

import collections
import contextlib
import sys
import wave
import webrtcvad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading and Testing Audios from Google Drive

In [ ]:
#Playing a sample audio
audio_path = "/content/drive/MyDrive/84-121123-0027.wav"
Audio(filename=audio_path)

## Reading from CSV file and storing in an array

In [ ]:
#Reading audio files and storing in an array

data_list = []
label_list = []

with open("/content/drive/MyDrive/Name_label_list.csv") as csv_file:
    data_from_csv = csv.reader(csv_file, delimiter=',')
    for i in data_from_csv:
        #Storing labels and the data in arrays
        
        data_list.append(i[0])
        label_list.append(i[1])

print(data_list[0])

/content/drive/MyDrive/Audios/Depp/almost.wav


## Removing Silence from Audio

In [ ]:
def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in range(8000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])


audio, sample_rate = read_wave(audio_path)
print(audio)
vad = webrtcvad.Vad(3)
frames = frame_generator(10, audio, sample_rate)
frames = list(frames)
segments = vad_collector(sample_rate, 30, 300, vad, frames)

# Segmenting the Voice audio and save it in list as bytes
concataudio = [segment for segment in segments]

joinedaudio = b"".join(concataudio)

write_wave("/content/drive/MyDrive/84-121123-0027-sil-rem.wav", joinedaudio, sample_rate)

#Audio(filename=audio_path)

def find_length(audio_name):
  with contextlib.closing(wave.open(audio_name,'r')) as f:
    frame_count = f.getnframes()
    rate = f.getframerate()
    duration = frame_count/float(rate)
    print(duration)

find_length("/content/drive/MyDrive/84-121123-0027.wav")
find_length("/content/drive/MyDrive/84-121123-0027-sil-rem.wav")
Audio(filename="/content/drive/MyDrive/84-121123-0027-sil-rem.wav")

b'\xca\xff\xc0\xff\x9f\xff\xa9\xff\xa8\xff\xbd\xff\xc0\xff\xbe\xff\xcc\xff\xd8\xff\xc9\xff\xb7\xff\xae\xff\x9d\xff\x90\xff\x8a\xff\x9d\xff\xa5\xff\x95\xff\x7f\xff\x89\xff\x81\xff\\\xfff\xffd\xffp\xff}\xffz\xffw\xffj\xffh\xfft\xffr\xffo\xffn\xffa\xffg\xffw\xffZ\xff`\xfft\xff\x84\xffm\xffg\xffj\xffj\xff`\xffV\xffm\xff~\xff\x82\xff\x90\xff\x95\xff\x8f\xff\x9d\xff\xa5\xff\xb9\xff\xc1\xff\xc3\xff\xc5\xff\xcf\xff\xc7\xff\xc4\xff\xcc\xff\xd6\xff\xcb\xff\xd6\xff\xde\xff\xcf\xff\xd3\xff\xd8\xff\xee\xff\xe2\xff\xe6\xff\xdc\xff\xe7\xff\xdd\xff\xd5\xff\xd1\xff\xd4\xff\xd6\xff\xe4\xff\xf8\xff\x08\x00\x12\x00\x17\x00"\x00\x19\x00\x16\x00\x00\x00\xe6\xff\xf4\xff\xec\xff\xe6\xff\xe0\xff\xdd\xff\xe9\xff\xe0\xff\xee\xff\xfe\xff\xf3\xff\xde\xff\xf6\xff\t\x00\x17\x00\x17\x00:\x00I\x009\x00O\x00o\x00t\x00U\x00k\x00h\x00[\x00I\x00H\x00>\x00,\x00 \x00\'\x003\x00B\x00V\x00c\x00}\x00\x87\x00\x9c\x00\x91\x00\x8e\x00\x8c\x00\x8c\x00t\x00x\x00m\x00s\x00p\x00\x82\x00p\x00^\x00^\x00R\x00S\x00B\x00T\x00O\x00R\x00V\x

## Hamming Windowing and FFT

In [ ]:
#Taping the voice signal at the end and begining to remove any discontinuities
#Also doing Fast Fourier Transform to convert audio from time domain to frequency domain
from scipy.fft import fft, fftfreq

## Converting audios to MFCC

In [ ]:
#Breaking the audios into frames and then calculating the mean MFCC values of each
import librosa
main_mfcc_array = [] #[mfcc values, label]

#To find the mean of each vector of the mfcc, then create an array (or list) of these means
def generate_Mean_Arr(audio):
    y, sr = librosa.load(audio)

    # Set the hop length; at 22050 Hz, 512 samples ~= 23ms
    hop_length = 512
    
    # Compute MFCC features from the raw signal
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=39) #39 mfcc values

    #To create an array with the mean mfcc value vectors
    meanArr = []
    for i in mfcc:
        tempSum = 0
        counter = 0
        for j in numpy.nditer(i):
            tempSum+=j
            counter+=1
        meanArr.append(tempSum/float(counter))

    return meanArr

print(generate_Mean_Arr("/content/drive/MyDrive/84-121123-0027.wav"))

for i in range(len(data_list)):
  try:
    main_mfcc_array.append((generate_Mean_Arr(data_list[i])))
  except ValueError:
    main_mfcc_array.append((main_mfcc_array[i-1]))

print(len(main_mfcc_array))

[-345.2248891959513, 124.08201140210144, -18.404135128385143, 40.56224743405679, -6.8208791499299215, 10.935003684427505, -6.139113939906421, -21.282054182282067, -5.0319855221008, 4.554102331960111, -3.3445744124570287, -4.585192370151443, 3.2875807739066003, -4.082904750971418, -5.698828931141617, -7.2028350043565705, -0.24519064305420207, -8.651561891730118, -4.691173713682289, -3.1905681986109653, -6.777765151813514, -6.8677197174918385, -7.661335061815448, -0.30624428200990633, -9.578296802994004, -2.095317802482978, -3.068981948427688, -6.840754428304228, -2.04012371902179, -4.889283013747151, -1.484188067285638, -1.749612924299742, 1.4454536197665042, 1.587644444932615, 3.8420439972017046, 5.478822456378686, 8.749426725215482, 12.287423633991327, 10.819460535183884]


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

90


## Training the Model

In [ ]:
#Using the MFCC values from above to train a basic neural network model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import layers

#Splitting the data into training and testing sets, then converting to numpy array
data_train,data_test,target_train,target_test = train_test_split(main_mfcc_array,label_list,test_size=0.2,stratify=label_list,random_state=4)
data_train = numpy.array(data_train)
#target_train = numpy.array(target_train)
print(target_train)

for i in range(len(target_train)):
  if (target_train[i] == 'Johnny Depp'):
    target_train[i] = 0
  elif (target_train[i] == 'Robert Downey Jr.'):
    target_train[i] = 1
  else:
    target_train[i] = 2

for i in range(len(target_test)):
  if (target_test[i] == 'Johnny Depp'):
    target_test[i] = 0
  elif (target_test[i] == 'Robert Downey Jr.'):
    target_test[i] = 1
  else:
    target_test[i] = 2

print(len(target_train))

target_test = numpy.array(target_test)
target_train = numpy.array(target_train)
data_test = numpy.array(data_test)

#Defining the model
model = Sequential()

model.add(layers.Dense(256, activation='relu', input_shape=(39,)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#Training and testing
model.fit(data_train, target_train,epochs=20, batch_size=128, validation_split=0.2)
score = model.evaluate(data_test, target_test, verbose=0)
# print(confusion_matrix(data_test, target_test))

print(score)

['Johnny Depp', 'Robert Downey Jr.', 'Robert Downey Jr.', 'Lady Gaga', 'Johnny Depp', 'Robert Downey Jr.', 'Robert Downey Jr.', 'Johnny Depp', 'Johnny Depp', 'Lady Gaga', 'Johnny Depp', 'Robert Downey Jr.', 'Lady Gaga', 'Robert Downey Jr.', 'Robert Downey Jr.', 'Lady Gaga', 'Johnny Depp', 'Lady Gaga', 'Lady Gaga', 'Johnny Depp', 'Robert Downey Jr.', 'Robert Downey Jr.', 'Lady Gaga', 'Lady Gaga', 'Johnny Depp', 'Lady Gaga', 'Johnny Depp', 'Johnny Depp', 'Robert Downey Jr.', 'Lady Gaga', 'Lady Gaga', 'Robert Downey Jr.', 'Johnny Depp', 'Lady Gaga', 'Robert Downey Jr.', 'Robert Downey Jr.', 'Johnny Depp', 'Lady Gaga', 'Lady Gaga', 'Lady Gaga', 'Johnny Depp', 'Robert Downey Jr.', 'Johnny Depp', 'Johnny Depp', 'Johnny Depp', 'Johnny Depp', 'Robert Downey Jr.', 'Robert Downey Jr.', 'Lady Gaga', 'Lady Gaga', 'Robert Downey Jr.', 'Lady Gaga', 'Lady Gaga', 'Johnny Depp', 'Robert Downey Jr.', 'Lady Gaga', 'Robert Downey Jr.', 'Lady Gaga', 'Robert Downey Jr.', 'Johnny Depp', 'Robert Downey Jr.', 